## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hilo,US,19.7297,-155.0900,87.28,77,75,10.36,broken clouds
1,1,Basoko,CD,1.2391,23.6160,72.30,93,100,2.84,light rain
2,2,Bubaque,GW,11.2833,-15.8333,80.13,77,100,9.82,overcast clouds
3,3,Hervey Bay,AU,-25.2986,152.8535,63.50,62,0,9.66,clear sky
4,4,Sorland,NO,67.6670,12.6934,49.86,79,14,13.09,few clouds


In [3]:
city_data_df.dtypes

City_ID                  int64
City                    object
Country                 object
Lat                    float64
Lng                    float64
Max Temp               float64
Humidity                 int64
Cloudiness               int64
Wind Speed             float64
Current Description     object
dtype: object

In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hilo,US,19.7297,-155.0900,87.28,77,75,10.36,broken clouds
2,2,Bubaque,GW,11.2833,-15.8333,80.13,77,100,9.82,overcast clouds
6,6,Kapaa,US,22.0752,-159.3190,85.98,64,40,16.11,scattered clouds
20,20,Hithadhoo,MV,-0.6000,73.0833,82.35,73,100,12.21,overcast clouds
24,24,Akot,IN,21.0964,77.0586,77.22,88,100,3.78,overcast clouds
37,37,Avarua,CK,-21.2078,-159.7750,77.05,69,75,20.71,broken clouds
48,48,Oistins,BB,13.0667,-59.5333,80.69,78,40,18.41,scattered clouds
55,55,Cabo San Lucas,MX,22.8909,-109.9124,88.11,51,20,14.97,few clouds
60,60,Beidao,CN,34.5686,105.8933,79.48,49,0,1.07,clear sky
61,61,Cayenne,GF,4.9333,-52.3333,76.08,1,0,0.00,clear sky


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                251
City                   251
Country                251
Lat                    251
Lng                    251
Max Temp               251
Humidity               251
Cloudiness             251
Wind Speed             251
Current Description    251
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Hilo,US,87.28,broken clouds,19.7297,-155.0900,
2,Bubaque,GW,80.13,overcast clouds,11.2833,-15.8333,
6,Kapaa,US,85.98,scattered clouds,22.0752,-159.3190,
20,Hithadhoo,MV,82.35,overcast clouds,-0.6000,73.0833,
24,Akot,IN,77.22,overcast clouds,21.0964,77.0586,
37,Avarua,CK,77.05,broken clouds,-21.2078,-159.7750,
48,Oistins,BB,80.69,scattered clouds,13.0667,-59.5333,
55,Cabo San Lucas,MX,88.11,few clouds,22.8909,-109.9124,
60,Beidao,CN,79.48,clear sky,34.5686,105.8933,
61,Cayenne,GF,76.08,clear sky,4.9333,-52.3333,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
hotel_df = hotel_df.replace(r'^s*$', float('NaN'), regex = True)
hotel_df.info()

#https://statisticsglobe.com/drop-rows-blank-values-from-pandas-dataframe-python

<class 'pandas.core.frame.DataFrame'>
Int64Index: 251 entries, 0 to 688
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City                 251 non-null    object 
 1   Country              251 non-null    object 
 2   Max Temp             251 non-null    float64
 3   Current Description  251 non-null    object 
 4   Lat                  251 non-null    float64
 5   Lng                  251 non-null    float64
 6   Hotel Name           239 non-null    object 
dtypes: float64(3), object(4)
memory usage: 23.8+ KB


In [11]:
# 7. Drop the rows where there in no Hotel Name.
clean_hotel_df = hotel_df.dropna(how="any")

In [12]:
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Hilo,US,87.28,broken clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
2,Bubaque,GW,80.13,overcast clouds,11.2833,-15.8333,"Casa Dora, Bubaque"
6,Kapaa,US,85.98,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
20,Hithadhoo,MV,82.35,overcast clouds,-0.6000,73.0833,Scoop Guest House
24,Akot,IN,77.22,overcast clouds,21.0964,77.0586,Shri PRASAD
37,Avarua,CK,77.05,broken clouds,-21.2078,-159.7750,Paradise Inn
48,Oistins,BB,80.69,scattered clouds,13.0667,-59.5333,Butterfly Beach Hotel
55,Cabo San Lucas,MX,88.11,few clouds,22.8909,-109.9124,Hotel Tesoro Los Cabos
60,Beidao,CN,79.48,clear sky,34.5686,105.8933,Shell Tianshui Maiji District Erma Road Walkin...
61,Cayenne,GF,76.08,clear sky,4.9333,-52.3333,Hôtel Le Dronmi


In [13]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))